In [ ]:
import datetime
import json
import requests
from requests.cookies import RequestsCookieJar
from dataclasses import dataclass
from tenacity import retry, stop_after_attempt, wait_fixed
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *


@dataclass
class ServerInfo:
    name: str
    base_url: str
    auth_cookie: RequestsCookieJar = None


class DataRetriever:
    secrets_scope = "NMIS"
    api_id_secret_key = "nmis_api_id"
    api_pwd_secret_key = "nmis_api_pwd"
    auth_endpoint = "/omk/opCharts/login?username={api_id}&password={api_pwd}"
    errors: list[Exception]

    def __init__(
        self,
        servers: list[ServerInfo],
        target_dir: str,
        data_endpoints: list[str],
        payload: dict = None,
        headers: dict = None,
        dbutils=None,
        power_automate_url: str = None,  # Add Power Automate URL
    ):
        self.servers = servers
        self.target_dir = target_dir
        self.dbutils = dbutils
        self.api_id = self.dbutils.secrets.get(
            self.secrets_scope, self.api_id_secret_key
        )
        self.api_pwd = self.dbutils.secrets.get(
            self.secrets_scope, self.api_pwd_secret_key
        )
        self.data_endpoints = data_endpoints
        self.payload = payload
        self.headers = headers
        self.power_automate_url = power_automate_url  # Add Power Automate URL

    def get_auth_cookie(self, server_url) -> RequestsCookieJar:
        url = f"{server_url}{self.auth_endpoint}".format(
            api_id=self.api_id, api_pwd=self.api_pwd
        )
        auth_response = requests.post(url)
        if auth_response.status_code == 200:
            return auth_response.cookies
        raise Exception(
            f"Auth call failed for url: {server_url}. Response: {auth_response}"
        )

    @staticmethod
    @retry(stop=stop_after_attempt(10), wait=wait_fixed(2))
    def make_api_call(
        api_url: str,
        cookies: RequestsCookieJar,
        payload: dict = None,
        headers: dict = None,
    ) -> any:
        response = requests.get(
            api_url, cookies=cookies, params=payload, headers=headers
        )

        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(
                f"Data call failed for url: {api_url}. Response: {response}"
            )

    @staticmethod
    def now() -> datetime:
        return datetime.datetime.now()

    def write_data(self, target_dir, server_name, data, schema=None, overwrite=False):
        currentDatetime = DataRetriever.now().strftime("%Y%m%d%H%M%S")
        target_filename = f"{target_dir}/{server_name}-{currentDatetime}.json"
        if schema is not None:
            data_json = json.dumps(data)
            df = spark.read.schema(schema).json(
                spark.sparkContext.parallelize([data_json])
            )
            # df.write.json(target_filename)
            df_string = df.toJSON().collect()
            df_str_combined = "[" + ",".join(df_string) + "]"
            self.dbutils.fs.put(target_filename, df_str_combined)
        else:  # No schema for retriever1
            self.dbutils.fs.put(target_filename, json.dumps(data, ensure_ascii=False))

    def send_teams_notification(self, error_message):
        # Prepare the payload with the 'error_message' property
        # Use HTML <br> tags for line breaks in Teams
        formatted_error_message = "Error occurred in script:<br><br>" + "<br><br>".join(
            error_message
        )
        message = {"error_message": formatted_error_message}
        print("Sending the following message to Teams:", message)  # Debug message
        try:
            response = requests.post(self.power_automate_url, json=message)
            if response.status_code not in [200, 202]:
                raise Exception(
                    f"Failed to send Teams notification via Power Automate. "
                    f"Status code: {response.status_code}, Response: {response.text}"
                )
            else:
                print(
                    f"Successfully sent notification to Teams. Status code: {response.status_code}"
                )
        except Exception as e:
            print(f"Exception occurred while sending notification: {e}")
            raise e

    def process_all_servers(self, schema=None):
        self.dbutils.fs.mkdirs(self.target_dir)
        errors = ["NMIS9 Fetch:"]
        for server in self.servers:
            for endpoint in self.data_endpoints:
                data_url = f"{server.base_url}{endpoint}"
                try:
                    server.auth_cookie = self.get_auth_cookie(server.base_url)
                    response_data = self.make_api_call(
                        data_url, server.auth_cookie, self.payload, self.headers
                    )
                    if schema is not None:  # Apply schema for all servers in retriever2
                        self.write_data(
                            self.target_dir, server.name, response_data, schema
                        )
                    else:
                        self.write_data(self.target_dir, server.name, response_data)
                except Exception as e:
                    error_message = f"Error occurred during server {server.name} for endpoint {endpoint}."
                    print(
                        f"Exception occurred while processing server: {server}. Exception: {e}"
                    )
                    errors.append(error_message)

        if len(errors) > 1:
            # Join error messages with two newlines for spacing
            self.send_teams_notification(errors)

In [ ]:
schema_interface = StructType(
    [
        StructField("_id", StringType(), False),
        StructField(
            "catchall",
            StructType(
                [
                    StructField("node_uuid", StringType(), False),
                ]
            ),
            False,
        ),
        StructField(
            "inventory",
            StructType(
                [
                    StructField(
                        "data",
                        StructType(
                            [
                                StructField("interface", StringType(), False),
                                StructField("ifDescr", StringType(), False),
                                StructField("Description", StringType(), False),
                                StructField("ifSpeed", StringType(), False),
                                StructField("collect", StringType(), False),
                                StructField("ifLastChange", StringType(), False),
                                StructField("ifOperStatus", StringType(), False),
                                StructField("ifType", StringType(), False),
                                StructField("ifIndex", StringType(), False),
                            ]
                        ),
                        False,
                    ),
                ]
            ),
            False,
        ),
        StructField(
            "latest_data",
            StructType(
                [
                    StructField(
                        "subconcepts",
                        StructType(
                            [
                                StructField(
                                    "interface",
                                    StructType(
                                        [
                                            StructField(
                                                "derived_data",
                                                StructType(
                                                    [
                                                        StructField(
                                                            "availability",
                                                            StringType(),
                                                            False,
                                                        ),
                                                        StructField(
                                                            "inputUtil",
                                                            StringType(),
                                                            False,
                                                        ),
                                                        StructField(
                                                            "outputUtil",
                                                            StringType(),
                                                            False,
                                                        ),
                                                        StructField(
                                                            "totalUtil",
                                                            StringType(),
                                                            False,
                                                        ),
                                                    ]
                                                ),
                                                False,
                                            ),
                                        ]
                                    ),
                                    False,
                                ),
                            ]
                        ),
                        False,
                    ),
                ]
            ),
            False,
        ),
        StructField("event_timestamp", TimestampType(), False),
        StructField("id", StringType(), False),
    ]
)

In [ ]:
power_automate_url = dbutils.secrets.get("NMIS", "power_automate_url")

retriever1 = DataRetriever(
    [
        ServerInfo("lpun90nms1", "http://lpun90nms1.jdnet.deere.com"),
        ServerInfo("lbei90nms1", "http://lbei90nms1.jdnet.deere.com"),
        ServerInfo("lbne90nms1", "http://lbne90nms1.jdnet.deere.com"),
        ServerInfo("lman90nms1", "http://lman90nms1.jdnet.deere.com"),
        ServerInfo("lman90nms2", "http://lman90nms2.jdnet.deere.com"),
        ServerInfo("lvin90nms1", "http://lvin90nms1.jdnet.deere.com"),
        ServerInfo("ldxx90nms40", "http://ldxx90nms40.dx.deere.com"),
        ServerInfo("ldxx90nms41", "http://ldxx90nms41.dx.deere.com"),
        ServerInfo("ldxx90nms42", "http://ldxx90nms42.dx.deere.com"),
        ServerInfo("ldxx90nms32", "http://ldxx90nms32.dx.deere.com"),
    ],
    "/mnt/edl/raw/nmis_dc_logs/nmis9/unprocessed",
    ["/omk/opCharts/nodes.json"],
    dbutils=dbutils,
    power_automate_url=power_automate_url,
)
retriever1.process_all_servers()

retriever2 = DataRetriever(
    [
        ServerInfo("lpun90nms1", "http://lpun90nms1.jdnet.deere.com"),
        ServerInfo("lbei90nms1", "http://lbei90nms1.jdnet.deere.com"),
        ServerInfo("lbne90nms1", "http://lbne90nms1.jdnet.deere.com"),
        ServerInfo("lman90nms1", "http://lman90nms1.jdnet.deere.com"),
        ServerInfo("lman90nms2", "http://lman90nms2.jdnet.deere.com"),
        ServerInfo("lvin90nms1", "http://lvin90nms1.jdnet.deere.com"),
        ServerInfo("ldxx90nms40", "http://ldxx90nms40.dx.deere.com"),
        ServerInfo("ldxx90nms41", "http://ldxx90nms41.dx.deere.com"),
        ServerInfo("ldxx90nms42", "http://ldxx90nms42.dx.deere.com"),
        ServerInfo("ldxx90nms32", "http://ldxx90nms32.dx.deere.com"),
    ],
    "/mnt/edl/raw/nmis_dc_logs/nmis9_interfaces/unprocessed",
    [
        '/omk/opCharts/components/inventories.json?search={"inventory.data_info.subconcept":"interface","enabled":0,"historic":0}',
        '/omk/opCharts/components/inventories.json?search={"inventory.data_info.subconcept":"interface","enabled":1,"historic":0}',
    ],
    payload={
        "catchall.data.name": "1",
        "inventory.data_info.subconcept": "interface",
    },
    headers={"Content-Type": "application/x-www-form-urlencoded"},
    dbutils=dbutils,
    power_automate_url=power_automate_url,
)
retriever2.process_all_servers(schema_interface)